In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os

from urllib.request import urlopen

# 출력화면 초기화 하는 모듈
from IPython.display import clear_output

- 네이버 영화 140자평 데이터 수집
- 소스보기 했을 때 원하는 데이터가 없었다.
- iframe 있는지 확인
- iframe의 src 속성의 주소를 요청해서 원하는 데이터가 있는지 확읺나다.

### 요청함수

In [2]:
def getSource(site):
    # 헤더 정보
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Safari/537.36',   
    }
    
    # 요청한다.
    response = requests.get(site, headers = header_info)
#     print(response.text)

    # bs4 객체 생성
    soup = bs4.BeautifulSoup(response.text,'lxml')
    
    return soup

### 한 페이지의 데이터를 수집해 저장하는 함수

In [158]:
# (인덱스, 값) 형태
k1 = enumerate([10,20,30,40,50])
list(k1)

[(0, 10), (1, 20), (2, 30), (3, 40), (4, 50)]

In [161]:
def getData1(soup):
    # ul태그를 가져온다.
    a1 = soup.select_one('body > div > div > div.score_result > ul')
    
    # li 태그를 가져온다.
    a2 = a1.select('li')
    
    # li 태그의 수만큼 반복하기 위해 값을 가져옴
    for a3 in a2:
        # 평점을 가져온다.
        a4 = a3.select_one('div.star_score > em')
        data1 = a4.text.strip()
        
        # 관람평 데이터
        a5 = a3.select('div.score_reple > p > span')
        data2 = a5[-1].text.strip()
      
        # 작성자 데이터
        a6 = a3.select_one('div.score_reple > dl > dt > em > a > span')
        data3 = a6.text.strip()
        
        # 작성일자 데이터
        a7 = a3.select('div.score_reple > dl > dt > em')
        data4 = a7[-1].text.strip() 
        print(data2)

In [162]:
site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=196051&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2'

soup = getSource(site)
getData(soup)


좋은스토리, 극장판다운 연출, 앞부분이 별로인 사람들도 입을 다물게하는 마지막부분. 살면서 애니영화보고 관객들 우는거처음본 명작 영화!!
우리들이 바라는 참된어른
말 그대로 영혼을 갈아 넣어버렸습니다..애니메이션을  이만큼 정밀하게 그릴 수 있다는 것에 놀라웠어요! 다시 보러 가고 싶어집니다.
저야말로 당신같은 분께서 낳아주셔서 영광이었습니다.
애니를 평소에 안보던 사람인데, 유일하게 원작완결까지 찾아보게한 만화.. 애니를 영화관에서 보니 좀 신기하기도 했는데, 전율도 배로 느껴지더라구요.. ㅋㅋ 만화로 볼 때도 감동적이었는데, 애니화된걸보니 작화가 어마무...
다 좋은데 단한가지 우울한점은 다음스토리가 도대체 언제쯤 애니로나올까...네요 ㅠㅠ얼른 규타로편도 애니나 극장판으로 나왔으면좋겠습니다
렌고쿠의 렌고쿠 그냥 미쳤다고밖에 말할 수 없다.
너무 감동적이고 재밌었습니다!!!
너무 재미있어서 태어나서 처음으로 별점줍니다.


## 강사님 풀이

### 영화 코드 수집함수

In [192]:
def getMovieCode():
    site = 'https://movie.naver.com/movie/running/current.nhn'
    soup = getSource(site)
    
    # ul태그를 가져옴
    a = soup.select_one('#content > div.article > div:nth-child(1) > div.lst_wrap > ul')
    
    # li태그를 가져온다.
    a1 = a.select('li')
    
    # 영화코드를 담을 리스트
    code_list = []
    
    # li 태그의 수만큼 반복한다.
    for a2 in a1:
        a3 = a2.select_one('dl > dt > a')
        
        # href 속성값을 가져온다.
        href = a3.attrs['href']
        # =를 기준으로 문자열 나누기
        data1 = href.split('=')[1]
        code_list.append(data1.strip())
    return code_list

### 한 페이지의 데이터를 수집해 저장하는 함수

In [203]:
def getData(soup, page):
    
    # 데이터를 담을 딕셔너리 생성
    data_dict ={
        '평점' : [],
        '네티즌평': [],
        '작성자' : [],
        '작성시간' : []
    }
    
    
    # ul태그를 가져온다.
    a1 = soup.select_one('body > div > div > div.score_result > ul')
    
    # li 태그를 가져온다.
    a2 = a1.select('li')

    for idx,a3 in enumerate(a2):
        # 평점을 가져온다.
        a4 = a3.select_one('div.star_score > em')
        data1 = a4.text.strip()
        
        # 관람평 데이터
        a5 = a3.select_one(f'#_filtered_ment_{idx}')
        data2 = a5.text.strip()

        # 작성자 데이터
        a6 = a3.select_one('div.score_reple > dl > dt > em > a > span')
        data3 = a6.text.strip()
        
        # 작성일자 데이터
        a7 = a3.select('div.score_reple > dl > dt > em')
        data4 = a7[-1].text.strip()
        
        # 딕셔너리에 추가
        data_dict['평점'].append(data1)
        data_dict['네티즌평'].append(data2)
        data_dict['작성자'].append(data3)
        data_dict['작성시간'].append(data4)
        
        
        print(data1)
        print(data2)
        print(data3)
        print(data4)
        print('-'*20)
    
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)

            
#     # 데이터 프레임 저장
#     if os.path.exists('Naver_moive.csv') == False:
#         df1.to_csv('Naver_moive.csv', encoding='utf-8-sig', index=False)
#     else:
#         df1.to_csv('Naver_moive.csv', encoding='utf-8-sig', index=False, header=False, mode='a')
    
    
    
    
    # 전체 페이지의 수를 계산한다. (갯수가 나와있는경우)
    a8 = soup.select_one('body > div > div > div.score_total > strong > em')
    data5 =  a8.text.strip()
    data5 = data5.replace(',','') # 쉼표제거
    total_page = int(data5) // 10
    
    if int(data5) % 10  > 0:
        total_page +=1
    
    # 테스트용
    total_page = 4
    
    if total_page > page :
        # 전체 페이지수가  현재 페이지번호보다 크면 True
        return True
    else:
        return False
        
        

### 결과확인

In [205]:
page = 1  

# 영화 코드들을 가져온다.
movie_code_list = getMovieCode()

# 테스트용 코드
movie_code_list = movie_code_list[:3]

# 실행부
for movie_code in movie_code_list:
    page = 1  
    while True:
        time.sleep(1)
        clear_output(wait=True)
        print(f'{movie_code} : {page} 수집중' )
        site = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        soup = getSource(site)
        chk  = getData(soup, page)
        if chk == True:
            page = page + 1
        else:
            print('수집완료')
            break

19335 : 4 수집중
9
아휘의 집에서 혼자 울부짖던 보영이 가장 기억에 남는다. 돌아올 곳 없는 보영의 방랑이 다시 시작될 것을 알아서 더 마음이 아팠다.
llilllilllil(niob****)
2017.03.14 02:01
--------------------
10
그냥 이게 인생같다. 인위적인 해피엔딩이 아닌 진짜 인생... 그리고 아...장국영....miss you much
별빛K(feel****)
2017.02.12 02:28
--------------------
10
왕가위 영화 주인공들 중 최고의 케미스트리가 돋보인 수작
무서워하구나 연어(ciph****)
2013.04.03 00:13
--------------------
8
장국영은 떠났지만, 부에노스 아이레스 어딘가에는 보영이 살아 있을것만 같다. 눈물겨운 아휘, 양조위와 푸른 장첸 모두 오랫동안 기억에 남을듯
ㅋㅋㅋ(7282****)
2014.06.13 02:16
--------------------
10
마치 다큐를 보는 듯한 미친 연기력 !!!
onod****
2013.12.02 01:06
--------------------
10
스무살 무렵 개봉당시 볼땐 답답하고 우울하기만 했는데 나이들어 다시보니  레알 감동.. 레슬리 그립소~~ ㅜㅠ
chic****
2016.05.06 05:43
--------------------
10
스크린 속 장국영은 늙지 않는다. 장첸도, 양조위도. 그때 그 시절의 지독하게 외로운 영화는 지금 봐도 아름다웠다.
Colorful Life(fina****)
2013.08.21 02:12
--------------------
9
굳이 동성애로 국한시키지 않아도 모든 면에서 뛰어난 최고의 사랑 영화
바람의언덕(sipj****)
2014.12.30 21:23
--------------------
10
왕가위 감독의 최고의 걸작...
iufe****
2013.04.02 09:54
--------------------
10
게이양화인줄 알고 봤는데 방황하는 우리들